<a href="https://colab.research.google.com/github/pushpixels/AgentGPT/blob/main/Cold_Call_Assist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**25% off** BlandAI: https://app.bland.ai/signup?ref=763d7133-0070-45e9-92ac-bf7c83080a62

In [ ]:
!pip install pandas
!pip install requests

In [ ]:
import requests
import pandas as pd

# Define your API key and headers for authentication
api_key = 'YOUR_BLANDAI_API_KEY'
headers = {
    "authorization": api_key,
    "Content-Type": "application/json"
}

# Load the call list from a CSV file
df = pd.read_csv('YOUR_FILEPATH')


# Define the payload for the batch call
batch_payload = {
    "base_prompt": "You're calling to speak with {{name}} about refinancing their mortgage down to six percent. Tell them it's never been a better time to refinance.  You're calling from Estate of Bliss Realty. If the person is interested, transfer the call to {{default}}.",
    "call_data": [
        {
            "phone_number": str(row['Phone Number']),
            "name": row['Name'],
        } for index, row in df.iterrows()
    ],
    #define transfer phone numbers
    "transfer_list": {
        "default": "15558457896",
        "sales": "1555552323"
    },
    #Define tough to pronounce words
    "pronunciation_guide": [
      {
        "word": "realty",
        "pronunciation": "real-tea",
      },
      {
        "word": "Estate",
        "pronunciation": "Es-state",
      }
    ],
    "model": "enhanced",
    "wait_for_greeting": False,
    "first_sentence": "Hi there, I'm calling about refinancing your mortgage down to six percent!  Are you the current home owner?",
    "voice": "maya",            #this changed on Bland AI's side, so I updated here - different from video
    "reduce_latency": True,
    "temperature": 0.7,
    "test_mode": False,
    "campaign_id": "Tech-at-Work",
    "label": "Testing-1"
}

# Sending the batch of calls
send_batch_url = "https://api.bland.ai/v1/batches"
response = requests.post(send_batch_url, json=batch_payload, headers=headers)
batch_response = response.json()

if batch_response.get('message') == 'success':
    print("Batch sent successfully")
    batch_id = batch_response.get('batch_id')
else:
    print("Failed to send batch")
    print(response.text)
    # If the batch fails to send, halt further execution
    raise Exception("Batch sending failed, halting execution.")

In [ ]:
# Analyze the batch calls and give a status of each
analyze_url = f"https://api.bland.ai/v1/batches/{batch_id}/analyze"
analyze_payload = {
    "goal": "Is customer interested in refinancing their mortgage",
    "questions": [
        ["Who answered the call?", "human or voicemail"],
        ["Was the customer interested in refinancing?", "yes or no"],
        ["Did they ask to be transferred?", "yes or no"],
        ["What reason did the customer have for not wanting to refinance?", "string"]
    ]
}

analyze_response = requests.post(analyze_url, json=analyze_payload, headers=headers)
analyze_result = analyze_response.json()

if analyze_result['status'] == 'success':
    print("Batch analyzed successfully")
    answers = analyze_result['answers']
    # Compile analysis results into a report
    report_data = []
    for call_id, analysis in answers.items():
        report_row = {
            "Call ID": call_id,
            "Answered By": analysis[0] if analysis[0] is not None else "N/A",
            "Interested in Refinancing": analysis[1] if analysis[1] is not None else "N/A",
            "Ask to be transferred": analysis[2] if analysis[2] is not None else "N/A",
            "Reason Not Interested": analysis[3] if analysis[3] is not None else "N/A"
        }
        report_data.append(report_row)

    # Convert report data into a DataFrame
    report_df = pd.DataFrame(report_data)

    # Use to_string for better alignment in the Colab output window, index=False removes the DataFrame index
    print(report_df.to_string(index=False))
        # Optionally save to CSV
    #report_df.to_csv('call_outcomes_report.csv', index=False)
else:
    print("Failed to analyze batch")
    print(analyze_response.text)

In [ ]:
# Use stop_call(call_id) function where necessary to stop calls
url = "https://api.bland.ai/v1/calls/{call_id}/stop"